# GROMACS molecular dynamics demonstration

Molecular dynamics (MD) is a versatile method for simulating the time-dependent behavior of molecular systems. This notebook demonstrates how MD simulations can be run using the GROMACS software to study the self-assembly of lipids. To accelerate the simulations, a coarse-grained (CG) model using the Martini force field is employed. In addition, parallel processing with multiple CPU cores is leveraged to demonstrate the principle of high-performance computing (HPC).

## Preparations

First, we need to make some choices regarding our MD simulation:

* What kind of lipid do we want to study and how many?
* What solvent do we want to use?
* How large (cubic) box do we want to simulate?

To provide these selections, run the configuration script below. For reference, W = water, EOL = ethanol and OD = octadecane. Lipid names are long and complicated, but feel free to search those up on the internet. The box size is the side length of the cubic cell in units of Å (= 0.1 nm).

In [ ]:
%run config.py

The input parameters are stored in the variables `lipid.value`, `solvent.value`, `nlip.value` and `box.value`, respectively.

## Building the solvated lipid system

The solvated lipid box will be constructed in the following. First, let's try inserting the selected lipids into a cubic simulation box of specified size. This is done using the GROMACS tool `insert-molecules` and the resulting system will be stored in a file named `lipids.gro`.

In [ ]:
!gmx_mpi insert-molecules -o lipids -ci ff22/{lipid.value}.gro -nmol {nlip.value} -try 200 -box {box.value}

GROMACS will dump quite a lot of output. The most important content is usually printed toward the bottom. Always make sure to check for any error messages. If you get an error, read the message and try to figure out what went wrong. A collection of common runtime errors is available in the [GROMACS documentation](https://manual.gromacs.org/current/user-guide/run-time-errors.html).

Next, we need to fill the `lipids.gro` system with the chosen solvent. As much solvent as will fit in the box containing the lipids is automatically added. This is done using the `solvate` tool of GROMACS and the resulting system will be stored in the file `solv.gro`.

In [ ]:
!gmx_mpi solvate -cp lipids.gro -cs ff22/{solvent.value}.gro -o solv

Great, now we have a box filled with lipids and solvent of your choice.

## The topology

To be able to perform the actual MD simulation, we need a file describing on which atoms and atom combinations (pairs, triples, quadruples) the different components of the force field should act and how (which parameters should be applied in each case). All this is contained in the topology file.

A template topology file is provided. However, before it can be used we need to fix some absolute paths in the file to point to the applied coarse-grained force field. At the same time, we need to specify the selected lipid type and solvent, as well as how many molecules of each were respectively added. To this end, check the previous `insert-molecules` and `solvate` outputs for the number of lipids and solvent molecules that were *de facto* added. Paste these numbers after the `{lipid.value}` and `{solvent.value}` variables in the command below (replace `N_LIPID_MOLS` and `N_SOLVENT_MOLS`).

The editing of the topology file `topol.top` is done automatically by the `init_files.sh` script below (feel free to have a look what it does under the hood with `!cat init_files.sh`). The script will also initialize input files needed for following energy minimization and MD steps.

**Note! If the number of lipid/solvent molecules in the topology does not match the structure file, the next steps will fail. So make sure to edit the numbers below carefully.**

In [ ]:
!bash init_files.sh {lipid.value} 50 {solvent.value} 1793

## Energy minimization

Now, let's copy a template recipe for performing *energy minimization*

In [ ]:
!gmx_mpi grompp -f em -c solv -p topol -o em -maxwarn 10

In [ ]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -v -s em

In [ ]:
!gmx_mpi make_ndx -f confout.gro < template/ndx_inp

In [ ]:
!gmx_mpi grompp -f md -c confout -p topol -maxwarn 10 -n index -o md

In [ ]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -deffnm md -v

In [ ]:
!gmx_mpi trjconv -s md -f md.xtc -o md_whole.xtc -pbc mol < template/trj_inp

In [ ]:
import MDAnalysis as mda
import nglview as nv

u = mda.Universe('md.gro', 'md_whole.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lipid.value)
view.add_point(solvent.value)
view.add_unitcell()
view

Cleanup

In [ ]:
!bash clean.sh